<a href="https://colab.research.google.com/github/TheoBacqueyrisse/graph-neural-networks/blob/main/Graph_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Graph Transformer Architecture**

In [1]:
# Let us first clone the GitHub repository
%%capture
!git clone https://github.com/TheoBacqueyrisse/Graph-Neural-Networks.git

In [2]:
# Install dependencies
%%capture
%cd /content/Graph-Neural-Networks
!pip install -r requirements.txt

In [3]:
from utils import *

## Graph Transformer Architecture

In [10]:
EMBEDDING_SIZE = 72
class GraphTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphTransformer, self).__init__()

        self.conv1 = TransformerConv(input_dim, hidden_dim, heads = 4)
        self.conv2 = TransformerConv(hidden_dim * 4, hidden_dim, heads = 4)

        self.pool = gap

        self.out = nn.Linear(hidden_dim * 4, output_dim)

    def forward(self, data, batch_index):

        x, edge_index, edge_attr = data.x.float(), data.edge_index, data.edge_attr

        max_edge_attr_size = 72

        if edge_attr.size(0) != max_edge_attr_size:
            if edge_attr.size(0) < max_edge_attr_size:
                edge_attr = torch.nn.functional.pad(edge_attr, (0, max_edge_attr_size - edge_attr.size(0)))
            else:
                edge_attr = edge_attr[:max_edge_attr_size]

        x = self.conv1(x, edge_index, edge_attr)
        x = F.sigmoid(x)
        x = F.dropout(x, p = 0.4)

        x = self.conv2(x, edge_index, edge_attr)
        x = F.sigmoid(x)
        x = F.dropout(x, p = 0.2)

        x = self.pool(x, batch_index)

        x = self.out(x)

        return x

model = GraphTransformer(input_dim = 1, hidden_dim = EMBEDDING_SIZE, output_dim = 1)

## Configuration

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

NUM_EPOCHS = 100

loss_function = L1Loss()

optimizer = Adam(params = model.parameters(), lr = 0.003)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, min_lr=0.00001)

## Load ZINC Data

In [12]:
NB_GRAPHS_PER_BATCH = 64

train = ZINC('/content/Graph-Neural-Networks/data', split = 'train')
train = train[train.y > -10] # Drop Outliers

val = ZINC('/content/Graph-Neural-Networks/data', split = 'val')

test = ZINC('/content/Graph-Neural-Networks/data', split = 'test')

train_loader = DataLoader(train,
                          batch_size = NB_GRAPHS_PER_BATCH,
                          shuffle = True)

val_loader = DataLoader(val,
                        batch_size = NB_GRAPHS_PER_BATCH,
                        shuffle = False)

test_loader = DataLoader(test,
                         batch_size = NB_GRAPHS_PER_BATCH,
                         shuffle = False)

print("Number of Batches in Train Loader :", len(train_loader))
print("Number of Batches in Val Loader :", len(val_loader))
print("Number of Batches in Test Loader :", len(test_loader))

Number of Batches in Train Loader : 3433
Number of Batches in Val Loader : 382
Number of Batches in Test Loader : 79


## Train and Test Functions 🚀

In [16]:
def train():
  for epoch in range(NUM_EPOCHS):
      model.train()
      total_loss = 0.0

      for batch_data in train_loader:

          # Use GPU
          batch_data.to(device)

          optimizer.zero_grad()
          predictions = model(batch_data, batch_data.batch)

          loss = loss_function(predictions, batch_data.y.float().view(-1, 1))
          loss.backward()
          optimizer.step()

          total_loss += loss.item()

      average_train_loss = total_loss / len(train_loader)

      model.eval()
      val_total_loss = 0.0
      with torch.no_grad():
          for val_batch_data in val_loader:
              val_batch_data.to(device)
              val_predictions = model(val_batch_data, val_batch_data.batch)
              val_loss = loss_function(val_predictions, val_batch_data.y.float().view(-1, 1))
              val_total_loss += val_loss.item()

          average_val_loss = val_total_loss / len(val_loader)
          scheduler.step(average_val_loss)

      print(f"Epoch {epoch + 1} -> Train Loss: {average_train_loss:.4f} - Val Loss: {average_val_loss:.4f}")

def test():
  model.eval()
  tot_test_loss = 0.0
  with torch.no_grad():

      for test_batch in test_loader:
          test_batch.to(device)

          test_predicitions = model(test_batch, test_batch.batch)
          test_loss = loss_function(test_predicitions, test_batch.y.view(-1, 1).float().view(-1, 1))

          tot_test_loss += test_loss.item()

      average_test_loss = tot_test_loss / len(test_loader)

  print(f"Test Loss: {average_test_loss:.4f}")

In [14]:
train()

Epoch 1 -> Train Loss: 0.9559 - Val Loss: 0.8533
Epoch 2 -> Train Loss: 0.7963 - Val Loss: 0.7862
Epoch 3 -> Train Loss: 0.7584 - Val Loss: 0.7619
Epoch 4 -> Train Loss: 0.7412 - Val Loss: 0.7602
Epoch 5 -> Train Loss: 0.7303 - Val Loss: 0.7468
Epoch 6 -> Train Loss: 0.7220 - Val Loss: 0.7242
Epoch 7 -> Train Loss: 0.7200 - Val Loss: 0.8063
Epoch 8 -> Train Loss: 0.7223 - Val Loss: 0.7393
Epoch 9 -> Train Loss: 0.7191 - Val Loss: 0.7240
Epoch 10 -> Train Loss: 0.7111 - Val Loss: 0.7260
Epoch 11 -> Train Loss: 0.7063 - Val Loss: 0.7494
Epoch 12 -> Train Loss: 0.7081 - Val Loss: 0.7230
Epoch 13 -> Train Loss: 0.7069 - Val Loss: 0.7152
Epoch 14 -> Train Loss: 0.7050 - Val Loss: 0.7134
Epoch 15 -> Train Loss: 0.6999 - Val Loss: 0.7114
Epoch 16 -> Train Loss: 0.6982 - Val Loss: 0.7329
Epoch 17 -> Train Loss: 0.6962 - Val Loss: 0.7081
Epoch 18 -> Train Loss: 0.6953 - Val Loss: 0.7034
Epoch 19 -> Train Loss: 0.6932 - Val Loss: 0.7164
Epoch 20 -> Train Loss: 0.6884 - Val Loss: 0.7132
Epoch 21 

In [17]:
test()

Test Loss: 0.6769
